In [1]:
import numpy as np
from skimage import io
from skimage import img_as_float,img_as_int
import matplotlib.pyplot as plt
import pickle,os
import imageio
from skimage.color import gray2rgb 

In [2]:
def find_len(arr):
    i = 0
    j = 0
    l = np.int(np.shape(arr)[0])
    k = np.int(l/2)
    for m in range(k-1):
        #print('m :',m,i,k,arr[k+1], arr[k])
        
        if arr[k+1] <= arr[k] :
            i = i + 1
            k = k + 1
        else :
            break
    k = np.int(l/2)
    for n in range(k-1):
        #print('n :',n,i,k,arr[k-1], arr[k])
        if arr[k-1] <= arr[k] :
            j = j + 1
            k = k - 1
        else :
            break
    #print(i,j,i+j)
    return i+j

In [3]:
def get_focal_spot(focal_plane_):
    focal_spot = np.zeros((200,200))
    x_,y_ = np.where(focal_plane_==np.max(focal_plane_))
    x_ = x_[0]
    y_ = y_[0]
    if (x_ > 100) & (y_>100):
        n = 100
        focal_spot = focal_plane_[x_-n:x_+n,y_-n:y_+n]
    else : 
        n = 50
        focal_spot[50:150,50:150] = focal_plane_[x_-n:x_+n,y_-n:y_+n]
    return focal_spot

In [4]:
angles = np.linspace(-1,1,50)
parameters = pickle.load(open('parameters_Au_417.pickle','rb'))
step_xy = parameters['step_xy']


In [6]:
focal_spot = []
spot = []
fwhm = []
max_intensity = []
n = 100
thickness = 5
os.chdir('/raid/home/sajid/ZP_simulations/Au_417_Chris_example/'+str(thickness)+'um_tilt/focal_spot')
for i in range(50):
    print(i)
    focal_spot_ = io.imread('focal_spot_'+str(i)+'.tiff')
    max_intensity.append(np.max(focal_spot_))
    focal_spot.append(focal_spot_)
    io.imsave('focal_spot__color_'+str(i)+'.tiff',gray2rgb(img_as_float(focal_spot_)))
    spotx = find_len(focal_spot[i][n,:])
    spoty = find_len(focal_spot[i][:,n])
    spot.append([spotx,spoty])
    
    fwhm1 = len(np.where(focal_spot[i][n,:]>np.max(focal_spot[i][n,:])*0.5)[0])
    fwhm2 = len(np.where(focal_spot[i][:,n]>np.max(focal_spot[i][:,n])*0.5)[0])
    fwhm.append([fwhm1,fwhm2])
    

0
1
2
3
4
5
6


/raid/home/sajid/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: focal_spot__color_0.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


IndexError: index 200 is out of bounds for axis 0 with size 200

In [ ]:
plt.clf()
f, axarr = plt.subplots(10,5)
k = 0
axis = []
max_plot_value = np.max(max_intensity)
    
for i in range(10):
    for j in range(5):
        axarr[i,j].imshow(focal_spot[k])
        axarr[i,j].axis('off')
        axarr[i,j].set_title(angles[k])
        axarr[i,j].title.set_y(1.01)
        #print(i,j,angles[k],k)
        axis.append(angles[k])
        k = k+1
f.set_size_inches(15, 15, forward=True)
f.subplots_adjust(wspace=0.5)
f.subplots_adjust(hspace=0.5)
f.suptitle('Tilt angles in degrees')
plt.show()

In [ ]:
fwhm = np.array(fwhm)
spot = np.array(spot)

In [ ]:
max_intensity = np.array(max_intensity)
rel_intensity = (max_intensity)/(np.max(max_intensity))
plt.plot(axis,rel_intensity,'*-')

np.save('max_intensity_'+str(thickness)+'_1deg.npy',max_intensity)

plt.ylabel('maximum intensity in focal plane (scaled)')
plt.xlabel('tilt anlge in degrees')
plt.title('scaled intensity vs tilt angle')
plt.rcParams["figure.figsize"] = (10,10)
plt.show()
#for i in range(50):
#    print(i,rel_intensity[i],spot[i])

In [ ]:
x_axis = np.linspace(-100*step_xy,100*step_xy,200)
max_loc = np.where(max_intensity==np.max(max_intensity))[0]
scaling = max_intensity[max_loc[0]]
np.save('max_intensity_1um_tilt_1deg_50_angles.npy',np.array(max_intensity))


for i in range(50):    
    j = (np.shape(focal_spot[i][0]))[0]
    k = int(j/2)
    focal_spot_ = focal_spot[i][k,:]/scaling
    plt.plot(x_axis,focal_spot_,label=str(angles[i]))
plt.legend(bbox_to_anchor=(1.20,1))
plt.rcParams["figure.figsize"] = (7,7)
plt.xlabel('distance')
plt.ylabel('scaled intensity at focal spot')
plt.title('Intensity along one axis of focal spot as a function of tilt angle')
plt.show()

In [ ]:
i = 23
var = focal_spot[i]
f,(ax1,ax2) = plt.subplots(1,2)
im1 = ax1.imshow(np.log(var))
f.colorbar(im1,ax=ax1)
im2 = ax2.plot(var[100,:])
f.subplots_adjust(right=1.75)
plt.show()